In [1]:
import pandas as pd
#import os
#pd.options.display.max_rows

#read in 39A leave time files
df39As = {}
keyList=[]
for file in os.listdir('Test/leavetimes/'):
    if file.startswith('39A'):
        keyList.append(file)
for key in keyList:
    df39As[key] = pd.read_csv('Test/leavetimes/'+file,names=['dayofservice','tripid','progrnumber','stopid','plannedtime',
                                                         'actual_arr','actual_dep'])

#Check key ring
df39As.keys()

#Let's look at one at a time
df39As['39A_40l']
#Interesting, only got 866003 rows instead of 1220760 rows of all. Seems like a limitation of dictionary values. Let's try 
#something else.

In [3]:
#read in 39A_40l
df39A_40l = pd.read_csv('Test/leavetimes/39A_40l',names=['dayofservice','tripid','progrnumber','stopid','plannedtime',
                                                         'actual_arr','actual_dep'])
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep
0,31-MAR-16,2901088,38,1661,60761,60789,60789
1,31-MAR-16,2901088,43,1807,61119,61096,61111
2,31-MAR-16,2901088,48,4464,62066,62137,62155
3,31-MAR-16,2901088,53,1873,62336,62426,62445
4,31-MAR-16,2901088,58,1878,62595,62656,62672
5,31-MAR-16,2901088,63,6109,62759,63007,63007
6,31-MAR-16,2901088,68,7011,62966,63300,63314
7,31-MAR-16,2901088,73,7162,63160,63543,63543
8,31-MAR-16,2901666,11,777,37151,37056,37056
9,31-MAR-16,2901666,16,783,37410,37308,37323


In [4]:
df39A_40l.dtypes

dayofservice    object
tripid           int64
progrnumber      int64
stopid           int64
plannedtime      int64
actual_arr       int64
actual_dep       int64
dtype: object

In [5]:
df39A_40l.dayofservice = df39A_40l.dayofservice.astype('datetime64[ns]')
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep
0,2016-03-31,2901088,38,1661,60761,60789,60789
1,2016-03-31,2901088,43,1807,61119,61096,61111
2,2016-03-31,2901088,48,4464,62066,62137,62155
3,2016-03-31,2901088,53,1873,62336,62426,62445
4,2016-03-31,2901088,58,1878,62595,62656,62672
5,2016-03-31,2901088,63,6109,62759,63007,63007
6,2016-03-31,2901088,68,7011,62966,63300,63314
7,2016-03-31,2901088,73,7162,63160,63543,63543
8,2016-03-31,2901666,11,777,37151,37056,37056
9,2016-03-31,2901666,16,783,37410,37308,37323


In [7]:
df39A_40l = df39A_40l.sort_values(['dayofservice','tripid','progrnumber'])
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep
667725,2016-01-09,2784393,1,767,73800,73722,73722
668086,2016-01-09,2784393,2,768,73925,73796,73815
291530,2016-01-09,2784393,3,769,73950,73839,73839
668453,2016-01-09,2784393,4,770,73992,73876,73876
668703,2016-01-09,2784393,5,771,74009,73883,73883
668760,2016-01-09,2784393,6,772,74044,73950,73950
669062,2016-01-09,2784393,7,773,74077,73986,73986
669388,2016-01-09,2784393,8,774,74116,74007,74007
669445,2016-01-09,2784393,9,775,74170,74044,74058
669783,2016-01-09,2784393,10,776,74209,74092,74092


In [8]:
df39A_40l.dayofservice[0]

Timestamp('2016-03-31 00:00:00')

In [9]:
df39A_40l.plannedtime[0]
#is 16.8 hours

60761

#In order to match with other time easily, change plannedtime, actual_arr, actual_dep to timestamp
#df['EndDate'] = pd.to_datetime(df['StartDate']) + pd.to_timedelta(df['duration'], unit='s')
print(df39A_40l.dayofservice[0]+pd.to_timedelta(df39A_40l.plannedtime[0],unit='s'))

#Consider this is more than one line, and will be used later, I wrote an function for it.
def changeToDatetime(df):
    '''A function that change plannedtime, actual_arr and actual_dep to datetime.'''
    df.plannedtime = df.dayofservice + pd.to_timedelta(df.plannedtime,unit='s')
    df.actual_arr = df.dayofservice + pd.to_timedelta(df.actual_arr,unit='s')
    df.actual_dep = df.dayofservice + pd.to_timedelta(df.actual_dep,unit='s')

changeToDatetime(df39A_40l)

In [12]:
#Why I want to change all the seconds to datetime format? Does that really make sense? How about create a dt for actuall_arr?
df39A_40l['datetime'] = df39A_40l.dayofservice + pd.to_timedelta(df39A_40l.actual_arr,unit='s')
#based on actual_arr
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep,datetime
667725,2016-01-09,2784393,1,767,73800,73722,73722,2016-01-09 20:28:42
668086,2016-01-09,2784393,2,768,73925,73796,73815,2016-01-09 20:29:56
291530,2016-01-09,2784393,3,769,73950,73839,73839,2016-01-09 20:30:39
668453,2016-01-09,2784393,4,770,73992,73876,73876,2016-01-09 20:31:16
668703,2016-01-09,2784393,5,771,74009,73883,73883,2016-01-09 20:31:23
668760,2016-01-09,2784393,6,772,74044,73950,73950,2016-01-09 20:32:30
669062,2016-01-09,2784393,7,773,74077,73986,73986,2016-01-09 20:33:06
669388,2016-01-09,2784393,8,774,74116,74007,74007,2016-01-09 20:33:27
669445,2016-01-09,2784393,9,775,74170,74044,74058,2016-01-09 20:34:04
669783,2016-01-09,2784393,10,776,74209,74092,74092,2016-01-09 20:34:52


In [15]:
#Extract time info from time columns: month, weekday
#df['YearMonth'] = df['ArrivalDate'].map(lambda x: 100*x.year + x.month)
df39A_40l['month'] = df39A_40l.dayofservice.map(lambda x: x.month)
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep,datetime,month
667725,2016-01-09,2784393,1,767,73800,73722,73722,2016-01-09 20:28:42,1
668086,2016-01-09,2784393,2,768,73925,73796,73815,2016-01-09 20:29:56,1
291530,2016-01-09,2784393,3,769,73950,73839,73839,2016-01-09 20:30:39,1
668453,2016-01-09,2784393,4,770,73992,73876,73876,2016-01-09 20:31:16,1
668703,2016-01-09,2784393,5,771,74009,73883,73883,2016-01-09 20:31:23,1
668760,2016-01-09,2784393,6,772,74044,73950,73950,2016-01-09 20:32:30,1
669062,2016-01-09,2784393,7,773,74077,73986,73986,2016-01-09 20:33:06,1
669388,2016-01-09,2784393,8,774,74116,74007,74007,2016-01-09 20:33:27,1
669445,2016-01-09,2784393,9,775,74170,74044,74058,2016-01-09 20:34:04,1
669783,2016-01-09,2784393,10,776,74209,74092,74092,2016-01-09 20:34:52,1


In [16]:
#df['weekday'] = df['Timestamp'].dt.dayofweek 
# 0-6 Mon-Sun
df39A_40l['weekday'] = df39A_40l.dayofservice.dt.dayofweek
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep,datetime,month,weekday
667725,2016-01-09,2784393,1,767,73800,73722,73722,2016-01-09 20:28:42,1,5
668086,2016-01-09,2784393,2,768,73925,73796,73815,2016-01-09 20:29:56,1,5
291530,2016-01-09,2784393,3,769,73950,73839,73839,2016-01-09 20:30:39,1,5
668453,2016-01-09,2784393,4,770,73992,73876,73876,2016-01-09 20:31:16,1,5
668703,2016-01-09,2784393,5,771,74009,73883,73883,2016-01-09 20:31:23,1,5
668760,2016-01-09,2784393,6,772,74044,73950,73950,2016-01-09 20:32:30,1,5
669062,2016-01-09,2784393,7,773,74077,73986,73986,2016-01-09 20:33:06,1,5
669388,2016-01-09,2784393,8,774,74116,74007,74007,2016-01-09 20:33:27,1,5
669445,2016-01-09,2784393,9,775,74170,74044,74058,2016-01-09 20:34:04,1,5
669783,2016-01-09,2784393,10,776,74209,74092,74092,2016-01-09 20:34:52,1,5


#extract time table for each stops
#First check if the planned time is the same everyday.
mask = (df39A_40l['dayofservice'] >= '2016-01-09') & (df39A_40l['dayofservice'] <= '2016-01-10')
df39A_40l.loc[mask].sort_values(['stopid','plannedtime'])

mask = (df39A_40l['dayofservice'] >= '2016-01-12') & (df39A_40l['dayofservice'] <= '2016-01-20')
df39A_40l.loc[mask].sort_values(['stopid','plannedtime'])

In [17]:
df39A_40l.sort_values(['stopid','plannedtime'])

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep,datetime,month,weekday
510882,2016-06-25,3260613,25,328,24402,24574,24602,2016-06-25 06:49:34,6,5
878167,2017-03-04,4391252,25,328,24440,24554,24604,2017-03-04 06:49:14,3,5
1149366,2017-03-11,4391252,25,328,24440,24494,24568,2017-03-11 06:48:14,3,5
1039918,2017-03-18,4391252,25,328,24440,24617,24679,2017-03-18 06:50:17,3,5
884326,2017-04-08,4532523,25,328,24449,24413,24445,2017-04-08 06:46:53,4,5
859345,2017-04-15,4580369,25,328,24449,24493,24532,2017-04-15 06:48:13,4,5
401633,2016-05-28,3038497,25,328,24464,24408,24440,2016-05-28 06:46:48,5,5
1016075,2017-06-10,5015335,25,328,24480,24632,24769,2017-06-10 06:50:32,6,5
891135,2017-06-17,5015335,25,328,24480,24662,24742,2017-06-17 06:51:02,6,5
667802,2016-01-09,2784478,25,328,24496,24510,24564,2016-01-09 06:48:30,1,5


In [ ]:
# all the 

In [ ]:
#Seems like most of the case, the planned time shall be the same, but some bus trip could be suppressed for some reason.

In [53]:
# Join weather info into the frame.
weather = pd.read_csv('CleanedWeather2016_17.csv',index_col=0)
weather

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
73487,01/01/2016 00:00,0,0.0,0,1.2,0,0.7,-0.2,6.0,90.0,1016.2
73488,01/01/2016 01:00,0,0.0,0,1.5,0,0.8,-0.7,5.8,85.0,1016.2
73489,01/01/2016 02:00,0,0.0,0,-0.2,0,-0.6,-1.3,5.6,92.0,1016.4
73490,01/01/2016 03:00,0,0.0,0,-0.7,0,-1.0,-1.4,5.5,95.0,1016.3
73491,01/01/2016 04:00,0,0.0,0,-0.3,0,-0.6,-1.2,5.6,94.0,1016.2
73492,01/01/2016 05:00,0,0.0,0,2.4,0,1.4,-0.3,6.0,82.0,1015.4
73493,01/01/2016 06:00,0,0.0,0,2.9,0,1.9,0.4,6.3,83.0,1014.6
73494,01/01/2016 07:00,0,0.0,0,2.5,0,1.8,0.7,6.4,88.0,1014.0
73495,01/01/2016 08:00,0,0.0,0,4.9,0,3.7,1.9,7.0,80.0,1013.0
73496,01/01/2016 09:00,0,0.0,0,6.3,0,5.0,3.3,7.7,81.0,1011.9


In [54]:
#Rename to match column name
#df.rename(index=str, columns={"A": "a", "B": "c"})
weather = weather.rename(index=str,columns={"date":"actual_arr"})

In [56]:
#got an error while merging, change datatype of actual_arr
weather.dtypes
#df39A_40l.dayofservice = df39A_40l.dayofservice.astype('datetime64[ns]')
# format='%d-%b-%y %H:%M:%S')
weather.actual_arr = pd.to_datetime(weather.actual_arr,format='%d/%m/%Y %H:%M')
weather

,actual_arr,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
73487,2016-01-01 00:00:00,0,0.0,0,1.2,0,0.7,-0.2,6.0,90.0,1016.2
73488,2016-01-01 01:00:00,0,0.0,0,1.5,0,0.8,-0.7,5.8,85.0,1016.2
73489,2016-01-01 02:00:00,0,0.0,0,-0.2,0,-0.6,-1.3,5.6,92.0,1016.4
73490,2016-01-01 03:00:00,0,0.0,0,-0.7,0,-1.0,-1.4,5.5,95.0,1016.3
73491,2016-01-01 04:00:00,0,0.0,0,-0.3,0,-0.6,-1.2,5.6,94.0,1016.2
73492,2016-01-01 05:00:00,0,0.0,0,2.4,0,1.4,-0.3,6.0,82.0,1015.4
73493,2016-01-01 06:00:00,0,0.0,0,2.9,0,1.9,0.4,6.3,83.0,1014.6
73494,2016-01-01 07:00:00,0,0.0,0,2.5,0,1.8,0.7,6.4,88.0,1014.0
73495,2016-01-01 08:00:00,0,0.0,0,4.9,0,3.7,1.9,7.0,80.0,1013.0
73496,2016-01-01 09:00:00,0,0.0,0,6.3,0,5.0,3.3,7.7,81.0,1011.9


In [57]:
#ValueError: left keys must be sorted
df39A_40l = df39A_40l.sort_values('actual_arr')
df39A_40l

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep
667717,2016-01-09,2784478,1,767,2016-01-09 06:30:00,2016-01-09 06:29:51,2016-01-09 06:29:51
667769,2016-01-09,2784478,2,768,2016-01-09 06:32:23,2016-01-09 06:31:26,2016-01-09 06:31:26
668137,2016-01-09,2784478,3,769,2016-01-09 06:32:48,2016-01-09 06:31:45,2016-01-09 06:31:45
668445,2016-01-09,2784478,4,770,2016-01-09 06:33:35,2016-01-09 06:32:15,2016-01-09 06:32:15
668498,2016-01-09,2784478,5,771,2016-01-09 06:33:54,2016-01-09 06:32:24,2016-01-09 06:32:24
668753,2016-01-09,2784478,6,772,2016-01-09 06:34:33,2016-01-09 06:33:05,2016-01-09 06:33:05
669054,2016-01-09,2784478,7,773,2016-01-09 06:34:57,2016-01-09 06:33:27,2016-01-09 06:33:44
669107,2016-01-09,2784478,8,774,2016-01-09 06:35:25,2016-01-09 06:34:03,2016-01-09 06:34:03
669438,2016-01-09,2784478,9,775,2016-01-09 06:36:04,2016-01-09 06:34:33,2016-01-09 06:34:33
669775,2016-01-09,2784478,10,776,2016-01-09 06:36:37,2016-01-09 06:34:55,2016-01-09 06:34:55


In [58]:
#ValueError: right keys must be sorted
weather = weather.sort_values('actual_arr')
weather

,actual_arr,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
73487,2016-01-01 00:00:00,0,0.0,0,1.2,0,0.7,-0.2,6.0,90.0,1016.2
73488,2016-01-01 01:00:00,0,0.0,0,1.5,0,0.8,-0.7,5.8,85.0,1016.2
73489,2016-01-01 02:00:00,0,0.0,0,-0.2,0,-0.6,-1.3,5.6,92.0,1016.4
73490,2016-01-01 03:00:00,0,0.0,0,-0.7,0,-1.0,-1.4,5.5,95.0,1016.3
73491,2016-01-01 04:00:00,0,0.0,0,-0.3,0,-0.6,-1.2,5.6,94.0,1016.2
73492,2016-01-01 05:00:00,0,0.0,0,2.4,0,1.4,-0.3,6.0,82.0,1015.4
73493,2016-01-01 06:00:00,0,0.0,0,2.9,0,1.9,0.4,6.3,83.0,1014.6
73494,2016-01-01 07:00:00,0,0.0,0,2.5,0,1.8,0.7,6.4,88.0,1014.0
73495,2016-01-01 08:00:00,0,0.0,0,4.9,0,3.7,1.9,7.0,80.0,1013.0
73496,2016-01-01 09:00:00,0,0.0,0,6.3,0,5.0,3.3,7.7,81.0,1011.9


In [59]:
#Merge
#pd.merge_asof(trades, quotes, on='time', by='ticker', tolerance=pd.Timedelta('2ms'))
pd.merge_asof(df39A_40l,weather,on='actual_arr',tolerance=pd.Timedelta('1h'))

,dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,2016-01-09,2784478,1,767,2016-01-09 06:30:00,2016-01-09 06:29:51,2016-01-09 06:29:51,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
1,2016-01-09,2784478,2,768,2016-01-09 06:32:23,2016-01-09 06:31:26,2016-01-09 06:31:26,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
2,2016-01-09,2784478,3,769,2016-01-09 06:32:48,2016-01-09 06:31:45,2016-01-09 06:31:45,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
3,2016-01-09,2784478,4,770,2016-01-09 06:33:35,2016-01-09 06:32:15,2016-01-09 06:32:15,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
4,2016-01-09,2784478,5,771,2016-01-09 06:33:54,2016-01-09 06:32:24,2016-01-09 06:32:24,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
5,2016-01-09,2784478,6,772,2016-01-09 06:34:33,2016-01-09 06:33:05,2016-01-09 06:33:05,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
6,2016-01-09,2784478,7,773,2016-01-09 06:34:57,2016-01-09 06:33:27,2016-01-09 06:33:44,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
7,2016-01-09,2784478,8,774,2016-01-09 06:35:25,2016-01-09 06:34:03,2016-01-09 06:34:03,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
8,2016-01-09,2784478,9,775,2016-01-09 06:36:04,2016-01-09 06:34:33,2016-01-09 06:34:33,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
9,2016-01-09,2784478,10,776,2016-01-09 06:36:37,2016-01-09 06:34:55,2016-01-09 06:34:55,0,0.0,0,2.2,0,2.2,2.1,7.1,99.0,985.9
